In [14]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json


view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')
cabinetColumns=view_cabinet.columns

view_cabinet=view_cabinet.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_cabinet.election_date=view_cabinet.election_date.apply(dateutil.parser.parse)


NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']

def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

def getLastCabinet(t):
    return t[t.start_date==t.start_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
Cabinets=view_cabinet.groupby('country_name_short').apply(getLastCabinet).reset_index(drop=True)

NationalElections=NationalElections[~NationalElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
EuroElections=EuroElections[~EuroElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
Cabinets=Cabinets[~Cabinets.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]


NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)
Cabinets.to_csv('../refdata/cabinets.csv',index=False)

def getPartyJson(code):
    topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total']
    
    nat = NationalElections[code==NationalElections.country_name_short] \
        .sort_values(['seats','vote_share','party_name_short'],ascending=False)
    r=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
    r['election_date']=str(r['election_date'])
    r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].dropna().to_dict() for x in nat.iterrows()]})

    return r


data={code:getPartyJson(code) for code in NationalElections['country_name_short'].drop_duplicates() if not code in ['AUS','CAN','ISR','NZL','JPN']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile,indent=4, default='')

In [20]:
Cabinets.groupby('cabinet_name')['party_name'].describe().unstack()


,count,unique,top,freq
cabinet_name,,,,
Anastasiades II,6,6,Kinima Sosialdimokraton EDEK,1
Bettel,6,6,Déi Lénk – La Gauche – Die Linke,1
Borisov II,8,8,Alternativa za balgarsko vazrazhdane,1
Bundesrat 2015,12,12,Lega dei Ticinesi,1
Butkevicius,9,9,Drąsos Kelias,1
Cerar,9,9,Slovenska demokratska stranka,1
Ciolos,9,9,no party affiliation,1
Costa,5,5,Partido Socialista,1
Fico III,8,8,Ľudová strana Naše Slovensko,1
